In [1]:
import pyforest


In [2]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [3]:
# Download the datasets
train_data, validation_data, test_data = tfds.load(name = "imdb_reviews",
                                                  split = ('train[:60%]','train[60%:]', 'test'),
                                                  as_supervised=True)

In [4]:
#Exploring the datasets
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [5]:
print(train_example_batch)

tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot de

In [6]:
print(train_labels_batch)

tf.Tensor([0 0 0 1 1 1 0 0 0 0], shape=(10,), dtype=int64)


In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape = [],
                          dtype = tf.string, trainable=True)

<IPython.core.display.Javascript object>

In [8]:
hub_layer(train_example_batch[:3])

<tf.Tensor: id=385, shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [9]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(tf.keras.layers.Dense(1))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer = 'adam',
             loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics = ['accuracy'])

<IPython.core.display.Javascript object>

In [13]:
data = model.fit(train_data.shuffle(10000).batch(512),
                epochs = 50,
                validation_data = validation_data.batch(512),
                verbose = 1)

Epoch 1/50
30/30 [==============================] - 9s 288ms/step - loss: 0.0751 - accuracy: 0.9815 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
30/30 [==============================] - 9s 285ms/step - loss: 0.0684 - accuracy: 0.9849 - val_loss: 0.3575 - val_accuracy: 0.8642
Epoch 3/50
30/30 [==============================] - 9s 287ms/step - loss: 0.0621 - accuracy: 0.9861 - val_loss: 0.3652 - val_accuracy: 0.8638
Epoch 4/50
30/30 [==============================] - 9s 287ms/step - loss: 0.0575 - accuracy: 0.9882 - val_loss: 0.3733 - val_accuracy: 0.8649
Epoch 5/50
30/30 [==============================] - 9s 286ms/step - loss: 0.0517 - accuracy: 0.9887 - val_loss: 0.3810 - val_accuracy: 0.8645
Epoch 6/50
30/30 [==============================] - 9s 290ms/step - loss: 0.0476 - accuracy: 0.9901 - val_loss: 0.3890 - val_accuracy: 0.8632
Epoch 7/50
30/30 [==============================] - 9s 289ms/step - loss: 0.0429 - accuracy: 0.9915 - val_loss: 0.3977 - val_accuracy: 0.863

In [15]:
result = model.evaluate(test_data.batch(512), verbose = 2)

for name, value in zip(model.metrics_names, result):
    print("%s : %.3f"%(name, value))

49/49 - 5s - loss: 0.8423 - accuracy: 0.8358
loss : 0.842
accuracy : 0.836
